<a href="https://colab.research.google.com/github/morales-cmd/Multimodal_biometric_verification_dev/blob/main/System_3modalny_show_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive to Colab notebook drive mounting.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing libraries.
import pandas as pd
import tensorflow_datasets as tfds
import os
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from scikitplot.metrics import plot_confusion_matrix, plot_roc
import tensorflow as tf
import gc
import cv2
import math
from google.colab.patches import cv2_imshow
from tensorflow.keras import datasets, layers, models,optimizers, metrics
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
import tensorflow as tf
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import scipy
import numpy as np
from numpy.polynomial.polynomial import Polynomial
import cv2
import librosa, librosa.display
from sklearn.naive_bayes import GaussianNB
from timeit import default_timer as timer
from random import random
from sklearn.neighbors import KNeighborsClassifier

# Signature feature extraction

In [ ]:

# euclidean distnace between 2 points (input 2 points)
def distance(curpoint,prevpoint):
  return np.sqrt((curpoint[0]-prevpoint[0])**2+(curpoint[1]-prevpoint[1])**2)

# return acc stats (min,max, mean, zerocrossings) stat (input non-stationary points with vel and acc)
def acceleration_stats(points):
  arr=np.array(points)
  arr=np.transpose(arr)
  zero_crossings = librosa.zero_crossings(arr[6], pad=False)
  return(min(arr[6]),max(arr[6]),np.mean(arr[6]),sum(zero_crossings))

# return vector with acc/dur stat (input non-stationary points with vel and acc)
def acceleration_by_duration(points):
  sum=0
  for i in range(1,len(points)):
    if(points[i][6]>0):
      sum+= points[i][4]-points[i-1][4]
  dur=points[len(points)-1][4]-points[0][4]
  return sum/dur

# return vector with all strokes durations (input non-stationary points)
def calc_vel_acc(points):
  points[0].append(0)  #0 velocity at 1st point
  points[0].append(0)  #0 acceleration at 1st point

  for i in range(1,len(points)):
    dist=distance(points[i],points[i-1])
    velocity= dist/(points[i][4]-points[i-1][4])
    points[i].append(velocity)
    acceleration=(points[i][5]-points[i-1][5])/(points[i][4]-points[i-1][4])
    points[i].append(acceleration)

# return vector with all strokes durations (input all points)
def stroke_dur(points):
  strokes_durs=[]
  start=points[0][4]
  for i in range(1,len(points)):
    if(points[i][3]>0 and points[i-1][3]==0):
      start=points[i][4]
    if(points[i][3]==0 and points[i-1][3]>0):
      strokes_durs.append(points[i][4]-start)
  return strokes_durs
def angle(v1,v2):
  return np.arccos(np.dot(np.array(v1),np.array(v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)))


def calc_dir_change_vel(points):
  points[0].append(0)  #0 velocity at 1st point
  points[1].append(0)  #0 acceleration at 1st point

  for i in range(2,len(points)):
    v1=[points[i][0]-points[i-1][0],points[i][1]-points[i-1][1]]
    v2=[points[i-1][0]-points[i-2][0],points[i-1][1]-points[i-2][1]]

    ang=angle(v1,v2);
    velocity= ang/(points[i][4]-points[i-1][4])
    # print(velocity)
    points[i].append(velocity)
    # acceleration=(points[i][5]-points[i-1][5])/(points[i][4]-points[i-1][4])
    # points[i].append(acceleration)


# return mean and std of dir change acceleration
def dir_vel_stats(points):
  arr=np.array(points)
  arr=np.transpose(arr)
  # print(np.nanmean(arr[7]))
  return arr[7]

# return vector with all strokes lengths (input all points)
def stroke_len(points):
  strokes_lens=[]
  sum=0
  if points[0][3]>0:
    start=1
  else:
    start=0
  for i in range(1,len(points)):
    if(points[i][3]>0 and points[i-1][3]==0):
      start=1
    if(points[i][3]==0 and points[i-1][3]>0):
      start=0
      strokes_lens.append(sum)
      sum=0
    if(start==1):
      sum+=distance(points[i],points[i-1])
  return strokes_lens

def average_stroke_press(points):
  strokes_press=[]
  it = 0
  sum = 0
  if points[0][3]>0:
    start=1
  else:
    start=0
  for i in range(1,len(points)):
    if(points[i][3]>0 and points[i-1][3]==0):
      it=0
      sum=0
      start=1
    if(points[i][3]==0 and points[i-1][3]>0):
      start=0
      strokes_press.append(sum/it)
    if(start==1):
      sum+=points[i][3]
      it+=1
  return strokes_press

#visualization of signature
def vizualization(points):
  im=np.zeros((200,200))
  for p in points:
    im[200-int(p[1]),int(p[0])]=255
  cv2.imshow(im)

# remove points with no x and y changes or no time changes
def remove_stationary_points(points):
  new_points=[]
  for i in range(1,len(points)):
    if((points[i][0]!=points[i-1][0] or points[i][1]!=points[i-1][1])and (points[i][4]!=points[i-1][4])):
      new_points.append(points[i])
  return new_points
def remove_noisy_points(points):
  new_points=[]
  for i in range(1,len(points)-1):
    if(~(points[i][3]==0 and points[i-1][3]>0 and points[i+1][3]>0)):
      new_points.append(points[i])
  return new_points

class Signature_features:
  def __init__(self, acc_by_dur, m_str_dur,std_str_dur,m_str_len,std_str_len, acc_stats,m_press,std_press):
    self.acc_by_dur = acc_by_dur
    self.m_str_dur = m_str_dur
    self.std_str_dur=std_str_dur
    self.m_str_len = m_str_len
    self.std_str_len=std_str_len
    self.acc_stats = acc_stats
    self.m_press= m_press
    self.std_press=std_press

# 0 acc_by_dur
# 1 m_str_dur
# 2 std_str_dur
# 3 m_str_len
# 4 std_str_len
# 5 m_str_len
# 6 min_acc
# 7 max_acc
# 8 mean_acc
# 9 zrc_acc
# 10 m_press
# 11 std_press
def get_stats(lines):
  points=[]
  for l in lines:
    a=l.replace("\n","")
    a=a.split(", ")
    a=[float(numeric_string) for numeric_string in a]
    points.append(a)
  points_reduced=remove_stationary_points(points)
  points_filtered=remove_noisy_points(points)
  calc_vel_acc(points_reduced)
  calc_dir_change_vel(points_reduced)
  return [acceleration_by_duration(points_reduced), np.mean(stroke_dur(points_filtered)),np.std(stroke_dur(points_filtered)), np.mean(stroke_len(points_filtered)), np.std(stroke_len(points_filtered)), acceleration_stats(points_reduced)[0],acceleration_stats(points_reduced)[1],acceleration_stats(points_reduced)[2],acceleration_stats(points_reduced)[3],np.mean(average_stroke_press(points_filtered)), np.std(average_stroke_press(points_filtered)),np.nanmean(dir_vel_stats(points_reduced)),np.nanstd(dir_vel_stats(points_reduced))]

def read_signature(path):
  with open(path) as f:
    lines = f.readlines()
  return lines

# Sets preparation modules

Splitting sets to train and test

In [ ]:
def split_mono_set(array, labels,seed):
    traincX = []
    trainY = []
    testcX = []
    testY = []
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    for train_index, test_index in sss.split(array, labels):
        for ind in train_index:
            traincX.append(array[ind])
            trainY.append(labels[ind])
        for ind in test_index:
            testcX.append(array[ind])
            testY.append(labels[ind])
    return traincX,trainY,testcX,testY

Merging of biometric features with cartesian product (with respect to owner)

In [ ]:
def merge_sets_cartesian(img_matrix,signature_matrix,wrist_matrix,image_labels,signature_labels,wrist_labels):
  merged_array=[]
  new_labels=[]
  for count ,sig in enumerate(signature_matrix):
    for count2, wrist in enumerate(wrist_matrix):
      if(wrist_labels[count2] == signature_labels[count]):
        merged_array.append([signature_matrix[count], wrist_matrix[count2]])
        wrist_labels=np.delete(wrist_labels, count2, 0)
        wrist_matrix=np.delete(wrist_matrix, count2, 0)
        new_labels.append(signature_labels[count])  
        break
  # print(img_matrix[0])
  merged_array2=[]
  new_labels2=[]
  for count, mer in enumerate(merged_array):
    for count2, img in enumerate(img_matrix):
      if(image_labels[count2] == new_labels[count]):
        merged_array2.append([img_matrix[count2], merged_array[count][0], merged_array[count][1]])
        new_labels2.append(image_labels[count2])
  # print(np.array(merged_array).shape)
  # print(merged_array[0])
  return merged_array2,new_labels2

Simple 1 - 1 merging of biometric features 

In [ ]:
def merge_sets_simple(img_matrix,signature_matrix,wrist_matrix,image_labels,signature_labels,wrist_labels):
  merged_array=[]
  new_labels=[]
  for count ,sig in enumerate(signature_matrix):
    for count2, img in enumerate(img_matrix):
      if(image_labels[count2]==signature_labels[count]):

        merged_array.append([img_matrix[count2], signature_matrix[count]])
        image_labels=np.delete(image_labels, count2, 0)
        img_matrix=np.delete(img_matrix, count2, 0)
        new_labels.append(signature_labels[count])
        break
  for count ,mer in enumerate(merged_array):
    for count2, wrist in enumerate(wrist_matrix):
      if(wrist_labels[count2]==new_labels[count]):

        merged_array[count].append(wrist_matrix[count2])
        wrist_labels=np.delete(wrist_labels, count2, 0)
        wrist_matrix=np.delete(wrist_matrix, count2, 0)
        break
  return merged_array,new_labels

In [ ]:
def split_sets_back(merged_matrix):
    img_matrix=[]
    sig_matrix=[]
    wrist_matrix=[]
    for  it in merged_matrix:
        img_matrix.append(it[0])
        sig_matrix.append(it[1])
        wrist_matrix.append(it[2])
    return img_matrix,sig_matrix, wrist_matrix

# wrist feature extraction

In [ ]:
def read_wrist_csv2(filename):
  dataframe = pd.read_csv(filename, skiprows=3)
  features=[]

  dataframe = dataframe.sort_values(by='HostTimestamp (ms)')
  dataframediff = dataframe[['X (dps)','Y (dps)','Z (dps)','HostTimestamp (ms)']].diff(periods=1)
  dataframediff=dataframediff[dataframediff['HostTimestamp (ms)']>15]
  dataframediff=dataframediff.head(80)
  fs=1/np.mean(dataframediff['HostTimestamp (ms)'])*1000
  
  rms=librosa.feature.rms(y=np.array(dataframediff['X (dps)']))
  features += [np.mean(rms)]
  rms=librosa.feature.rms(y=np.array(dataframediff['Y (dps)']))
  features += [np.mean(rms)]
  # print(np.std(rms))
  rms=librosa.feature.rms(y=np.array(dataframediff['Z (dps)']))
  features += [np.mean(rms)]

  zrc=librosa.feature.zero_crossing_rate(y=np.array(dataframediff['X (dps)']))
  features += [np.mean(zrc)]
  zrc=librosa.feature.zero_crossing_rate(y=np.array(dataframediff['Y (dps)']))
  features += [np.mean(zrc)]
  zrc=librosa.feature.zero_crossing_rate(y=np.array(dataframediff['Z (dps)']))
  features += [np.mean(zrc)]

  return features

# Decision module


Decision module of biometric system

In [ ]:
def decision_module(class_model,X, Y):
  X=np.array(X)
  Y=np.array(Y)
  dim=X.ndim
  if (dim==1):
    X = np.array([X])
    Y = np.array([Y])
  res = np.array(class_model.predict_proba(X))
  res_matrix = np.array([np.argmax(res,axis=1),np.max(res,axis=1)])
  max_cond = np.array(Y) == res_matrix[0]
  thresh_cond = decision_threshold < res_matrix[1]
  test_res = thresh_cond * max_cond
  # print(Y)
  # print(res_matrix)
  if(dim==1):
    return test_res[0]
  else:
    return test_res

# FAR, FRR test


In [ ]:
def FRR_test(class_model, testX, testY):
  res = decision_module(class_model, testX, testY)
  stat = np.sum(res)/res.shape[0]
  stat = 1-stat
  return stat

In [ ]:
def FAR_test(class_model, testX, testY):
  # add=np.random.random_integers(1, np.max(np.array(testY)), size = np.array(testY).shape[0])
  # testY = (testY+ add) % (np.max(np.array(testY))+1)
  testX_f=[]
  testY_f=[]
  for i in range(len(testY)):
    for j in range(np.max(np.array(testY))):
      testX_f.append(testX[i])
      testY_f.append((testY[i]+j+1) % (np.max(np.array(testY))+1))
  # print(testY_f)
  # print(testY)
  testX_f=np.array(testX_f)
  testY_f=np.array(testY_f)
  res = decision_module(class_model, testX_f, testY_f)
  stat = np.sum(res)/res.shape[0]
  return stat

# First model

model parameters (PCA components, decision odule threshold)

In [ ]:
seed=26
decision_threshold=0.3
comp_img=20
comp_sig=8
comp_wrist=4

Reading data + feature extraction

In [ ]:
root="/content/drive/MyDrive/sem3_biom_bazdana"
labels = []
image_matrix = []
image_labels = []
wrist_matrix=[]
signature_features_matrix=[]
signature_labels=[]
wrist_labels=[]
for user in os.listdir(f'{root}'):
    labels.append(user)
    for filename in os.listdir(f'{root}/{user}/faces'):

        img = cv2.imread(f'{root}/{user}/faces/{filename}')
        img = cv2.resize(img, (100, 100))
        image_matrix.append(np.array(img))
        image_labels.append(labels.index(user))
    for filename in os.listdir(f'{root}/{user}/signatures'):
        signature=read_signature(f'{root}/{user}/signatures/{filename}')
        stats = get_stats(signature)
        if stats[1]>0 and stats[0]<1:
            signature_features_matrix.append(stats)
            signature_labels.append(labels.index(user))
    for filename in os.listdir(f'{root}/{user}/wrist_gyroscope'):
        wrist_matrix.append(read_wrist_csv2(f'{root}/{user}/wrist_gyroscope/{filename}'))
        wrist_labels.append(labels.index(user))
IMG_SIZE = (100, 100)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
inputs = tf.keras.Input(shape=(100,100, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
outputs = global_average_layer(x)
model = tf.keras.Model(inputs, outputs)

image_matrix=model.predict(np.array(image_matrix))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


9420800/9406464 [==============================] - 0s 0us/step


In [ ]:


train_sig_X, train_sig_Y, test_sig_X, test_sig_Y = split_mono_set(signature_features_matrix, signature_labels,seed)
train_img_X, train_img_Y, test_img_X, test_img_Y = split_mono_set(image_matrix, image_labels,seed)
train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y = split_mono_set(wrist_matrix, wrist_labels,seed)

print(train_img_X[0].shape)

# train_img_X=model.predict(np.array(train_img_X))
# train_img_X=np.array(train_img_X)
pca = PCA(svd_solver="randomized",n_components=comp_img, whiten=True)
train_img_X = pca.fit_transform(np.array(train_img_X))


scaler_sig = StandardScaler()
train_sig_X=scaler_sig.fit_transform(train_sig_X)
pca_sig = PCA(svd_solver="randomized", n_components=comp_sig, whiten=True)
train_sig_X=pca_sig.fit_transform(train_sig_X)
# train_sig_X=np.fliplr(train_sig_X)

scaler_wrist= StandardScaler()
train_wrist_X=scaler_wrist.fit_transform(train_wrist_X)
pca_wrist = PCA(svd_solver="randomized", n_components=comp_wrist, whiten=True)
train_wrist_X=pca_wrist.fit_transform(train_wrist_X)
# train_wrist_X=np.fliplr(train_wrist_X)

test_img_X=pca.transform(np.array(test_img_X))

test_sig_X = scaler_sig.transform(np.array(test_sig_X))
test_sig_X = pca_sig.transform(test_sig_X)
# test_sig_X = np.fliplr(test_sig_X)

test_wrist_X = scaler_wrist.transform(np.array(test_wrist_X))
test_wrist_X = pca_wrist.transform(test_wrist_X)
# test_sig_X = np.fliplr(test_sig_X)

merged_trainX, merged_trainY = merge_sets_cartesian(train_img_X, train_sig_X, train_wrist_X, train_img_Y,
                                                     train_sig_Y,train_wrist_Y)
train_img_X, train_sig_X, train_wrist_X = split_sets_back(merged_trainX)

merged_testX, merged_testY = merge_sets_simple(test_img_X, test_sig_X, test_wrist_X, test_img_Y,
                                                     test_sig_Y,test_wrist_Y)
test_img_X, test_sig_X, test_wrist_X = split_sets_back(merged_testX)


trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X)),axis=1)
trainY=merged_trainY
testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X)),axis=1)
testY=merged_testY


(1280,)


In [ ]:
trainX.shape

(653, 28)

In [ ]:
model = SVC(kernel="rbf", C=30, gamma="auto",break_ties=False, random_state=3,decision_function_shape='ovr',probability=True)
model.fit(trainX, trainY)

SVC(C=30, gamma='auto', probability=True, random_state=3)

In [ ]:
predictions = model.predict(testX)

In [ ]:
res=classification_report(testY, predictions, target_names=labels)
print(res)

              precision    recall  f1-score   support

     Mateusz       1.00      1.00      1.00         2
      Szymon       1.00      1.00      1.00         2
       Pawel       1.00      1.00      1.00         2
       Lukas       1.00      1.00      1.00         3
       Kowal       1.00      1.00      1.00         2
    Gigakiek       1.00      1.00      1.00         2

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



# Gradient Descent dimension reduction modules

In [ ]:
from timeit import default_timer as timer
from random import random

In [ ]:
def gradient_descent(gradient, start ,minx , maxx, learn_rate, stop_time, n_iter):
    vector = start
    start_time = timer()
    for _ in range(n_iter):
        diff = -learn_rate * gradient(vector)
        vector += diff
        if vector < minx:
          vector=minx
        elif vector > maxx:
          vector=maxx
        if ((timer()-start_time)*1000>stop_time):
          break
    return vector

In [ ]:
def gradient_descent_fit_transform(data,pol_deg, learn_rate=0.1, stop_time = 30, n_iter=200):
  # pca_int = PCA(svd_solver="randomized",n_components=pol_deg+1, whiten=True)
  # data=pca_int.fit_transform(data)
  vectors=[]
  for vec in data:
    polynomials=[]
    for i in range(math.ceil(len(vec)/(pol_deg+1))):
      points=vec[i*(pol_deg+1):i*(pol_deg+1)+pol_deg]
      xrange=np.arange(len(points))
      lagrange_interpolation = scipy.interpolate.lagrange(xrange, points)
      grad = np.polyder(lagrange_interpolation, m=1)
      # print(str(min(xrange))+"   "+str(max(xrange)))
      x=gradient_descent(gradient=grad, start=random()*pol_deg,minx=min(xrange),maxx=max(xrange), learn_rate=learn_rate,n_iter=n_iter,stop_time=stop_time)
      polynomials.append(x)
      polynomials.append(lagrange_interpolation(x))
    vectors.append(np.array(polynomials))
      # Polynomial(lagrange_interpolation.coef[::-1]).coef
  return np.array(vectors)

# pipeline1

In [ ]:
# def gradient_descent_transform(data,pol_deg,pca_int,learn_rate,n_iter,stop_time):
#   data=pca_int.transform(data)
#   polynomials=[]
#   for vec in data:
#     xrange=np.arange(len(vec))
#     lagrange_interpolation = scipy.interpolate.lagrange(xrange, vec)
#     grad = np.polyder(lagrange_interpolation, m=1)
#     x=gradient_descent(gradient=grad, start=random()*pol_deg,minx=min(xrange),maxx=max(xrange), learn_rate=learn_rate,n_iter=n_iter,stop_time=stop_time)
#     polynomials.append([x,lagrange_interpolation(x)])
#     # Polynomial(lagrange_interpolation.coef[::-1]).coef
#   return np.array(polynomials)

In [ ]:
def feature_extraction1():
  root="/content/drive/MyDrive/sem3_biom_bazdana"
  global labels
  labels = []
  image_matrix = []
  image_labels = []
  wrist_matrix=[]
  signature_features_matrix=[]
  signature_labels=[]
  wrist_labels=[]
  for user in os.listdir(f'{root}'):
      labels.append(user)
      for filename in os.listdir(f'{root}/{user}/faces'):

          img = cv2.imread(f'{root}/{user}/faces/{filename}')
          # print(img.shape)
          img = cv2.resize(img, (120, 120))
          # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          # img = img.flatten()
          image_matrix.append(np.array(img))
          image_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/signatures'):
          signature=read_signature(f'{root}/{user}/signatures/{filename}')
          # print(img.shape)
          stats = get_stats(signature)
          # if stats[1]>0 and stats[0]<1:
          signature_features_matrix.append(stats)
          signature_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/wrist_gyroscope'):
          # print(f'{root}/{user}/wrist_gyroscope/{filename}')
          wrist_matrix.append(read_wrist_csv2(f'{root}/{user}/wrist_gyroscope/{filename}'))
          # print(img.shape)
          wrist_labels.append(labels.index(user))
         
  IMG_SIZE = (120, 120)
  IMG_SHAPE = IMG_SIZE + (3,)
  base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
  preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  inputs = tf.keras.Input(shape=(120,120, 3))
  x = preprocess_input(inputs)
  x = base_model(x, training=False)
  outputs = global_average_layer(x)
  model = tf.keras.Model(inputs, outputs)

  image_matrix=model.predict(np.array(image_matrix))
  global seed
  train_sig_X, train_sig_Y, test_sig_X, test_sig_Y = split_mono_set(signature_features_matrix, signature_labels,seed)
  train_img_X, train_img_Y, test_img_X, test_img_Y = split_mono_set(image_matrix, image_labels,seed)
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y = split_mono_set(wrist_matrix, wrist_labels,seed)
  return train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, train_img_X, train_img_Y, test_img_X, test_img_Y, train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y

In [ ]:
def transformations_concatenation1(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                     train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                     train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y):
  pca = PCA(svd_solver="randomized",n_components=comp_img, whiten=True)
  # scaler_img = StandardScaler()
  train_img_X = pca.fit_transform(np.array(train_img_X))
  # train_img_X = scaler_img.fit_transform(np.array(train_img_X))
  # print(train_img_X[0])

  scaler_sig = StandardScaler() #
  train_sig_X=scaler_sig.fit_transform(train_sig_X)
  pca_sig = PCA(svd_solver="randomized", n_components=comp_sig, whiten=True)
  train_sig_X=pca_sig.fit_transform(train_sig_X)
  # train_sig_X=np.fliplr(train_sig_X)

  scaler_wrist= StandardScaler()
  train_wrist_X=scaler_wrist.fit_transform(train_wrist_X)
  pca_wrist = PCA(svd_solver="randomized", n_components=comp_wrist, whiten=True)
  train_wrist_X=pca_wrist.fit_transform(train_wrist_X)
  # train_wrist_X=np.fliplr(train_wrist_X)

  test_img_X=pca.transform(np.array(test_img_X))
  # test_img_X = scaler_img.transform(np.array(train_img_X))

  test_sig_X = scaler_sig.transform(np.array(test_sig_X))
  test_sig_X = pca_sig.transform(test_sig_X)
  # test_sig_X = np.fliplr(test_sig_X)

  test_wrist_X = scaler_wrist.transform(np.array(test_wrist_X))
  test_wrist_X = pca_wrist.transform(test_wrist_X)
  # test_sig_X = np.fliplr(test_sig_X)

  merged_trainX, merged_trainY = merge_sets_cartesian(train_img_X, train_sig_X, train_wrist_X, train_img_Y,
                                                      train_sig_Y,train_wrist_Y)
  train_img_X, train_sig_X, train_wrist_X = split_sets_back(merged_trainX)

  merged_testX, merged_testY = merge_sets_simple(test_img_X, test_sig_X, test_wrist_X, test_img_Y,
                                                      test_sig_Y,test_wrist_Y)
  test_img_X, test_sig_X, test_wrist_X = split_sets_back(merged_testX)

  trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X),np.array(train_wrist_X)),axis=1)
  trainY=merged_trainY
  testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X),np.array(test_wrist_X)),axis=1)
  testY=merged_testY
  return trainX, trainY, testX, testY

In [ ]:
def training_testing_report1(trainX, trainY, testX, testY):
  class_model = SVC(kernel="rbf", C=30, gamma="auto",break_ties=False, random_state=3,decision_function_shape='ovr',probability=True)
  class_model.fit(trainX, trainY)
  predictions = class_model.predict(testX)
  res=classification_report(testY, predictions, target_names=labels)
  # print(res)
  return class_model,f1_score(testY, predictions,average='micro')

call pipeline1

In [ ]:
seed=20
decision_threshold=0.3
comp_img=20
comp_sig=7
comp_wrist=4

In [ ]:
res_arr=[]
for i in range(10):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction1()
  (trainX, trainY, testX, testY)=transformations_concatenation1(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report1(trainX, trainY, testX, testY)
  res_arr.append(res)
print(np.mean(res_arr))
print(np.std(res_arr))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


              precision    recall  f1-score   support

     Mateusz       0.67      1.00      0.80         2
      Szymon       1.00      1.00      1.00         2
       Pawel       1.00      1.00      1.00         2
       Lukas       1.00      1.00      1.00         3
       Kowal       1.00      0.50      0.67         2

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.89        11
weighted avg       0.94      0.91      0.90        11



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


              precision    recall  f1-score   support

     Mateusz       1.00      1.00      1.00         2
      Szymon       1.00      1.00      1.00         2
       Pawel       0.67      1.00      0.80         2
       Lukas       1.00      0.67      0.80         3
       Kowal       1.00      1.00      1.00         2

    accuracy                           0.91        11
   macro avg       0.93      0.93      0.92        11
weighted avg       0.94      0.91      0.91        11



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


...

              precision    recall  f1-score   support

     Mateusz       1.00      1.00      1.00         2
      Szymon       1.00      1.00      1.00         2
       Pawel       0.67      1.00      0.80         2
       Lukas       1.00      0.67      0.80         3
       Kowal       1.00      1.00      1.00         2

    accuracy                           0.91        11
   macro avg       0.93      0.93      0.92        11
weighted avg       0.94      0.91      0.91        11



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


              precision    recall  f1-score   support

     Mateusz       1.00      1.00      1.00         2
      Szymon       1.00      1.00      1.00         2
       Pawel       1.00      1.00      1.00         2
       Lukas       1.00      1.00      1.00         3
       Kowal       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


              precision    recall  f1-score   support

     Mateusz       1.00      1.00      1.00         2
      Szymon       1.00      1.00      1.00         2
       Pawel       1.00      1.00      1.00         2
       Lukas       1.00      1.00      1.00         3
       Kowal       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

0.9181818181818182
0.06363636363636363


In [ ]:
FRR_test(class_model, testX, testY)

0.0

In [ ]:
FAR_test(class_model, testX, testY)

0.022727272727272728

# pipeline2

In [ ]:
def feature_extraction2():
  root="/content/drive/MyDrive/sem3_biom_bazdana"
  global labels
  labels = []
  image_matrix = []
  image_labels = []
  wrist_matrix=[]
  signature_features_matrix=[]
  signature_labels=[]
  wrist_labels=[]
  for user in os.listdir(f'{root}'):
      labels.append(user)
      for filename in os.listdir(f'{root}/{user}/faces'):

          img = cv2.imread(f'{root}/{user}/faces/{filename}')
          # print(img.shape)
          img = cv2.resize(img, (120, 120))
          # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          # img = img.flatten()
          image_matrix.append(np.array(img))
          image_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/signatures'):
          signature=read_signature(f'{root}/{user}/signatures/{filename}')
          # print(img.shape)
          stats = get_stats(signature)
          if stats[1]>0 and stats[0]<1:
              signature_features_matrix.append(stats)
              signature_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/wrist_gyroscope'):
          # print(f'{root}/{user}/wrist_gyroscope/{filename}')
          wrist_matrix.append(read_wrist_csv2(f'{root}/{user}/wrist_gyroscope/{filename}'))
          # print(img.shape)
          wrist_labels.append(labels.index(user))
         
  IMG_SIZE = (120, 120)
  IMG_SHAPE = IMG_SIZE + (3,)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
  preprocess_input = tf.keras.applications.vgg16.preprocess_input
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  inputs = tf.keras.Input(shape=(120,120, 3))
  x = preprocess_input(inputs)
  x = base_model(x, training=False)
  outputs = global_average_layer(x)
  model = tf.keras.Model(inputs, outputs)

  image_matrix=model.predict(np.array(image_matrix))
  global seed
  train_sig_X, train_sig_Y, test_sig_X, test_sig_Y = split_mono_set(signature_features_matrix, signature_labels,seed)
  train_img_X, train_img_Y, test_img_X, test_img_Y = split_mono_set(image_matrix, image_labels,seed)
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y = split_mono_set(wrist_matrix, wrist_labels,seed)
  return train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, train_img_X, train_img_Y, test_img_X, test_img_Y, train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y

In [ ]:
def training_testing_report2(trainX, trainY, testX, testY):
  class_model = GaussianNB()
  class_model.fit(trainX, trainY)
  predictions = class_model.predict(testX)
  res=classification_report(testY, predictions, target_names=labels)

  # print(res)
  # print(f1_score(testY, predictions,average='micro'))
  return class_model,f1_score(testY, predictions,average='micro')

call pipeline 2

In [ ]:
seed=20
decision_threshold=0.6
# comp_discard=3
comp_img=24
comp_sig=8
comp_wrist=4

In [ ]:

res_arr=[]
for i in range(10):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation1(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report2(trainX, trainY, testX, testY)
  res_arr.append(res)
print(np.mean(res_arr))
print(np.std(res_arr))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45:

0.9636363636363636
0.06030226891555271


In [ ]:
FRR_test(class_model, testX, testY)

0.09090909090909094

In [ ]:
FAR_test(class_model, testX, testY)

0.022727272727272728

# pipeline3

In [ ]:
seed=27
decision_threshold=0.5
# comp_discard=3
comp_img=20
comp_sig=8
comp_wrist=4

In [ ]:
def feature_extraction3():
  root="/content/drive/MyDrive/sem3_biom_bazdana"
  global labels
  labels = []
  image_matrix = []
  image_labels = []
  wrist_matrix=[]
  signature_features_matrix=[]
  signature_labels=[]
  wrist_labels=[]
  for user in os.listdir(f'{root}'):
      labels.append(user)
      for filename in os.listdir(f'{root}/{user}/faces'):

          img = cv2.imread(f'{root}/{user}/faces/{filename}')
          # print(img.shape)
          img = cv2.resize(img, (120, 120))
          img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          img = img.flatten()
          image_matrix.append(np.array(img))
          image_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/signatures'):
          signature=read_signature(f'{root}/{user}/signatures/{filename}')
          # print(img.shape)
          stats = get_stats(signature)
          if stats[1]>0 and stats[0]<1:
              signature_features_matrix.append(stats)
              signature_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/wrist_gyroscope'):
          # print(f'{root}/{user}/wrist_gyroscope/{filename}')
          wrist_matrix.append(read_wrist_csv2(f'{root}/{user}/wrist_gyroscope/{filename}'))
          # print(img.shape)
          wrist_labels.append(labels.index(user))
         

  train_sig_X, train_sig_Y, test_sig_X, test_sig_Y = split_mono_set(signature_features_matrix, signature_labels,seed)
  train_img_X, train_img_Y, test_img_X, test_img_Y = split_mono_set(image_matrix, image_labels,seed)
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y = split_mono_set(wrist_matrix, wrist_labels,seed)
  return train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, train_img_X, train_img_Y, test_img_X, test_img_Y, train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y

In [ ]:
def training_testing_report3(trainX, trainY, testX, testY):
  class_model = KNeighborsClassifier(n_neighbors=3)
  class_model.fit(trainX, trainY)
  predictions = class_model.predict(testX)
  res=classification_report(testY, predictions, target_names=labels)
  print(res)
  return class_model,f1_score(testY, predictions,average='micro')

In [ ]:


res_arr=[]
for i in range(10):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction3()
  (trainX, trainY, testX, testY)=transformations_concatenation1(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report3(trainX, trainY, testX, testY)
  res_arr.append(res)
print(np.mean(res_arr))
print(np.std(res_arr))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


              precision    recall  f1-score   support

     Mateusz       0.50      1.00      0.67         2
      Szymon       1.00      1.00      1.00         2
       Pawel       1.00      0.50      0.67         2
       Lukas       1.00      0.67      0.80         3
       Kowal       1.00      1.00      1.00         2

    accuracy                           0.82        11
   macro avg       0.90      0.83      0.83        11
weighted avg       0.91      0.82      0.82        11



In [ ]:
FRR_test(class_model, testX, testY)

0.18181818181818177

In [ ]:
FAR_test(class_model, testX, testY)

0.045454545454545456

# Gradient descent tests


In [ ]:
def transformations_concatenation_gradient_descent(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                     train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                     train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y):
  pca = PCA(svd_solver="randomized",n_components=comp_img, whiten=True)
  # scaler_img = StandardScaler()
  train_img_X = pca.fit_transform(np.array(train_img_X))
  # train_img_X = scaler_img.fit_transform(np.array(train_img_X))


  scaler_sig = StandardScaler() #
  train_sig_X=scaler_sig.fit_transform(train_sig_X)
  pca_sig = PCA(svd_solver="randomized", n_components=comp_sig, whiten=True)
  train_sig_X=pca_sig.fit_transform(train_sig_X)
  # train_sig_X=np.fliplr(train_sig_X)

  scaler_wrist= StandardScaler()
  train_wrist_X=scaler_wrist.fit_transform(train_wrist_X)
  pca_wrist = PCA(svd_solver="randomized", n_components=comp_wrist, whiten=True)
  train_wrist_X=pca_wrist.fit_transform(train_wrist_X)
  # train_wrist_X=np.fliplr(train_wrist_X)

  test_img_X=pca.transform(np.array(test_img_X))
  # test_img_X = scaler_img.transform(np.array(train_img_X))

  test_sig_X = scaler_sig.transform(np.array(test_sig_X))
  test_sig_X = pca_sig.transform(test_sig_X)
  # test_sig_X = np.fliplr(test_sig_X)

  test_wrist_X = scaler_wrist.transform(np.array(test_wrist_X))
  test_wrist_X = pca_wrist.transform(test_wrist_X)
  # test_sig_X = np.fliplr(test_sig_X)

  merged_trainX, merged_trainY = merge_sets_cartesian(train_img_X, train_sig_X, train_wrist_X, train_img_Y,
                                                      train_sig_Y,train_wrist_Y)
  train_img_X, train_sig_X, train_wrist_X = split_sets_back(merged_trainX)

  merged_testX, merged_testY = merge_sets_simple(test_img_X, test_sig_X, test_wrist_X, test_img_Y,
                                                      test_sig_Y,test_wrist_Y)
  test_img_X, test_sig_X, test_wrist_X = split_sets_back(merged_testX)

  #####

  train_img_X= gradient_descent_fit_transform(np.array(train_img_X),pol_deg,learn_rate,n_iter,stop_time)
  test_img_X = gradient_descent_fit_transform(test_img_X,pol_deg,learn_rate,n_iter,stop_time)

  train_sig_X = gradient_descent_fit_transform(np.array(train_sig_X),pol_deg,learn_rate,n_iter,stop_time)
  test_sig_X = gradient_descent_fit_transform(test_sig_X,pol_deg,learn_rate,n_iter,stop_time)

  train_wrist_X = gradient_descent_fit_transform(np.array(train_wrist_X),pol_deg,learn_rate,n_iter,stop_time)
  test_wrist_X = gradient_descent_fit_transform(test_wrist_X,pol_deg,learn_rate,n_iter,stop_time)
  #####

  if(mode == 'all'):
    trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X),np.array(train_wrist_X)),axis=1)
    trainY=merged_trainY
    testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X),np.array(test_wrist_X)),axis=1)
    testY=merged_testY
  elif mode == 'face':
    trainX=np.array(train_img_X)
    trainY=merged_trainY
    testX=np.array(test_img_X)
    testY=merged_testY
  elif mode == 'sig':
    trainX=np.array(train_sig_X)
    trainY=merged_trainY
    testX=np.array(test_sig_X)
    testY=merged_testY
  elif mode == 'wrist':
    trainX=np.array(train_wrist_X)
    trainY=merged_trainY
    testX=np.array(test_wrist_X)
    testY=merged_testY
  elif mode == 'face_sig':
    trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X)),axis=1)
    trainY=merged_trainY
    testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X)),axis=1)
    testY=merged_testY
  
  return trainX, trainY, testX, testY

In [ ]:
seed=27
decision_threshold=0.3

comp_img=24
comp_sig=9
comp_wrist=6

In [ ]:
pol_deg=2
learn_rate=0.05
n_iter=100
stop_time=20
mode='all'

In [ ]:
res_arr=[]
for i in range(10):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation_gradient_descent(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model, res= training_testing_report2(trainX, trainY, testX, testY)
  res_arr.append(res)
print(np.mean(res_arr))
print(np.std(res_arr))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45:

0.8818181818181818
0.07100226978096956


# RMSE evaluation


In [ ]:
def MSE(class_model,X, Y):
  X=np.array(X)
  Y=np.array(Y)
  dim=X.ndim
  if (dim==1):
    X = np.array([X])
    Y = np.array([Y]) 
  res = np.array(class_model.predict_proba(X))
  res_matrix=[]
  for i in range(len(Y)-1):
    res_matrix.append(1-res[i,Y[i]])
  
  res_matrix = np.array(res_matrix)

  # max_cond = np.array(Y) == res_matrix[0]
  return np.mean(np.square(res_matrix))

In [ ]:
def RMSE(class_model,X, Y):
  X=np.array(X)
  Y=np.array(Y)
  dim=X.ndim
  if (dim==1):
    X = np.array([X])
    Y = np.array([Y]) 
  res = np.array(class_model.predict_proba(X))
  res_matrix=[]
  for i in range(len(Y)-1):
    res_matrix.append(1-res[i,Y[i]])
  
  res_matrix = np.array(res_matrix)

  # max_cond = np.array(Y) == res_matrix[0]
  return np.mean(np.square(res_matrix))**(1/2)

In [ ]:
def MAE(class_model,X, Y):
  X=np.array(X)
  Y=np.array(Y)
  dim=X.ndim
  if (dim==1):
    X = np.array([X])
    Y = np.array([Y]) 
  res = np.array(class_model.predict_proba(X))
  res_matrix=[]
  for i in range(len(Y)-1):
    res_matrix.append(1-res[i,Y[i]])
  
  res_matrix = np.array(res_matrix)

  # max_cond = np.array(Y) == res_matrix[0]
  return np.mean(res_matrix)

In [ ]:
def FRR_test(class_model, testX, testY):
  res = decision_module(class_model, testX, testY)
  stat = np.sum(res)/res.shape[0]
  stat = 1-stat
  return stat

In [ ]:
def FAR_test(class_model, testX, testY):
  # add=np.random.random_integers(1, np.max(np.array(testY)), size = np.array(testY).shape[0])
  # testY = (testY+ add) % (np.max(np.array(testY))+1)
  testX_f=[]
  testY_f=[]
  for i in range(len(testY)):
    for j in range(np.max(np.array(testY))):
      testX_f.append(testX[i])
      testY_f.append((testY[i]+j+1) % (np.max(np.array(testY))+1))
  # print(testY_f)
  # print(testY)
  testX_f=np.array(testX_f)
  testY_f=np.array(testY_f)
  res = decision_module(class_model, testX_f, testY_f)
  stat = np.sum(res)/res.shape[0]
  return stat

In [ ]:
def evaluation_results_print():
  print(f'liczba rekordów w zbiorze treningowym: {len(trainX)}')
  print(f'liczba rekordów w zbiorze testowym: {len(testX)}')
  print('')
  print(f'średni f1-score dla {test_count} podziałów: {np.mean(f1_arr)}')
  print(f'odchylenie standardowe f1-score dla {test_count} podziałów: {np.std(f1_arr)}')
  print('')
  print(f'średni MSE dla {test_count} podziałów: {np.mean(MSE_arr)}')
  print(f'odchylenie standardowe MSE dla {test_count} podziałów: {np.std(MSE_arr)}')
  print('')
  print(f'średni RMSE dla {test_count} podziałów: {np.mean(RMSE_arr)}')
  print(f'odchylenie standardowe RMSE dla {test_count} podziałów: {np.std(RMSE_arr)}')
  print('')
  print(f'średni MAE dla {test_count} podziałów: {np.mean(MAE_arr)}')
  print(f'odchylenie standardowe MAE dla {test_count} podziałów: {np.std(MAE_arr)}')

# RMSE for multimodal system

In [ ]:
seed=20
decision_threshold=0.6
# comp_discard=3
comp_img=20
comp_sig=7
comp_wrist=4

In [ ]:
MSE_arr = []
RMSE_arr = []
MAE_arr = []
f1_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation1(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report2(trainX, trainY, testX, testY)

  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45:

In [ ]:
evaluation_results_print()

liczba rekordów w zbiorze treningowym: 653
liczba rekordów w zbiorze testowym: 13

średni f1-score dla 10 podziałów: 0.9461538461538461
odchylenie standardowe f1-score dla 10 podziałów: 0.06007884366082041

średni MSE dla 10 podziałów: 0.04680927488525505
odchylenie standardowe MSE dla 10 podziałów: 0.05450637473324295

średni RMSE dla 10 podziałów: 0.15318029965711968
odchylenie standardowe RMSE dla 10 podziałów: 0.1527909378275102

średni MAE dla 10 podziałów: 0.052295757009930645
odchylenie standardowe MAE dla 10 podziałów: 0.0578015537207285


# RMSE for subsystems

In [ ]:
def transformations_concatenation2(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                     train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                     train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y):
  pca = PCA(svd_solver="randomized",n_components=comp_img, whiten=True)
  # scaler_img = StandardScaler()
  train_img_X = pca.fit_transform(np.array(train_img_X))
  # train_img_X = scaler_img.fit_transform(np.array(train_img_X))


  scaler_sig = StandardScaler() #
  train_sig_X=scaler_sig.fit_transform(train_sig_X)
  pca_sig = PCA(svd_solver="randomized", n_components=comp_sig, whiten=True)
  train_sig_X=pca_sig.fit_transform(train_sig_X)
  # train_sig_X=np.fliplr(train_sig_X)

  scaler_wrist= StandardScaler()
  train_wrist_X=scaler_wrist.fit_transform(train_wrist_X)
  pca_wrist = PCA(svd_solver="randomized", n_components=comp_wrist, whiten=True)
  train_wrist_X=pca_wrist.fit_transform(train_wrist_X)
  # train_wrist_X=np.fliplr(train_wrist_X)

  test_img_X=pca.transform(np.array(test_img_X))
  # test_img_X = scaler_img.transform(np.array(train_img_X))

  test_sig_X = scaler_sig.transform(np.array(test_sig_X))
  test_sig_X = pca_sig.transform(test_sig_X)
  # test_sig_X = np.fliplr(test_sig_X)

  test_wrist_X = scaler_wrist.transform(np.array(test_wrist_X))
  test_wrist_X = pca_wrist.transform(test_wrist_X)
  # test_sig_X = np.fliplr(test_sig_X)

  merged_trainX, merged_trainY = merge_sets_simple(train_img_X, train_sig_X, train_wrist_X, train_img_Y,
                                                      train_sig_Y,train_wrist_Y)
  train_img_X, train_sig_X, train_wrist_X = split_sets_back(merged_trainX)

  merged_testX, merged_testY = merge_sets_simple(test_img_X, test_sig_X, test_wrist_X, test_img_Y,
                                                      test_sig_Y,test_wrist_Y)
  test_img_X, test_sig_X, test_wrist_X = split_sets_back(merged_testX)

  #####

  if(mode == 'all'):
    trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X),np.array(train_wrist_X)),axis=1)
    trainY=merged_trainY
    testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X),np.array(test_wrist_X)),axis=1)
    testY=merged_testY
  elif mode == 'face':
    trainX=np.array(train_img_X)
    trainY=merged_trainY
    testX=np.array(test_img_X)
    testY=merged_testY
  elif mode == 'sig':
    trainX=np.array(train_sig_X)
    trainY=merged_trainY
    testX=np.array(test_sig_X)
    testY=merged_testY
  elif mode == 'wrist':
    trainX=np.array(train_wrist_X)
    trainY=merged_trainY
    testX=np.array(test_wrist_X)
    testY=merged_testY
  elif mode == 'face_sig':
    trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X)),axis=1)
    trainY=merged_trainY
    testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X)),axis=1)
    testY=merged_testY

    #####
  return trainX, trainY, testX, testY

In [ ]:
def eval_results():
  # seed=20
  global seed
  MSE_arr = []
  RMSE_arr = []
  MAE_arr = []
  test_count=10
  for i in range(test_count):
    seed+=1
    (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
    train_img_X, train_img_Y, test_img_X, test_img_Y, 
    train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
    (trainX, trainY, testX, testY)=transformations_concatenation2(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                    train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                    train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
    class_model,res=training_testing_report2(trainX, trainY, testX, testY)
    MSE_arr.append(MSE(class_model, testX, testY))
    RMSE_arr.append(RMSE(class_model, testX, testY))
    MAE_arr.append(MAE(class_model, testX, testY))
  return MSE_arr, RMSE_arr, MAE_arr

For face


In [ ]:
seed=10
decision_threshold=0.6
# comp_discard=3
comp_img=20
comp_sig=7
comp_wrist=4
mode = 'face'

In [ ]:
MSE_arr = []
RMSE_arr = []
MAE_arr = []
f1_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation2(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report2(trainX, trainY, testX, testY)
  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45:

In [ ]:
evaluation_results_print()

liczba rekordów w zbiorze treningowym: 50
liczba rekordów w zbiorze testowym: 13

średni f1-score dla 10 podziałów: 0.9461538461538461
odchylenie standardowe f1-score dla 10 podziałów: 0.0692307692307692

średni MSE dla 10 podziałów: 0.0555206584755077
odchylenie standardowe MSE dla 10 podziałów: 0.06390745428481089

średni RMSE dla 10 podziałów: 0.17529877583866482
odchylenie standardowe RMSE dla 10 podziałów: 0.157451572443635

średni MAE dla 10 podziałów: 0.06634714613445082
odchylenie standardowe MAE dla 10 podziałów: 0.07126095393702972


For signature



In [ ]:
seed=20
decision_threshold=0.6
# comp_discard=3
comp_img=20
comp_sig=7
comp_wrist=4
mode = 'sig'

In [ ]:
MSE_arr = []
RMSE_arr = []
MAE_arr = []
f1_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation2(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report1(trainX, trainY, testX, testY)
  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

In [ ]:
evaluation_results_print()

liczba rekordów w zbiorze treningowym: 50
liczba rekordów w zbiorze testowym: 13

średni f1-score dla 10 podziałów: 0.7923076923076924
odchylenie standardowe f1-score dla 10 podziałów: 0.10348941574672083

średni MSE dla 10 podziałów: 0.254617758494677
odchylenie standardowe MSE dla 10 podziałów: 0.06318374047546532

średni RMSE dla 10 podziałów: 0.5004522662797216
odchylenie standardowe RMSE dla 10 podziałów: 0.06453903989189572

średni MAE dla 10 podziałów: 0.4561532896520747
odchylenie standardowe MAE dla 10 podziałów: 0.059893586884900166


For wrist



In [ ]:
seed=10
decision_threshold=0.6
# comp_discard=3
comp_img=20
comp_sig=7
comp_wrist=6
mode = 'wrist'

In [ ]:
MSE_arr = []
RMSE_arr = []
MAE_arr = []
f1_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y, 
  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)=feature_extraction2()
  (trainX, trainY, testX, testY)=transformations_concatenation2(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y, 
                                  train_wrist_X, train_wrist_Y, test_wrist_X, test_wrist_Y)
  class_model,res=training_testing_report1(trainX, trainY, testX, testY)
  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

In [ ]:
evaluation_results_print()

liczba rekordów w zbiorze treningowym: 50
liczba rekordów w zbiorze testowym: 13

średni f1-score dla 10 podziałów: 0.576923076923077
odchylenie standardowe f1-score dla 10 podziałów: 0.15858098560067924

średni MSE dla 10 podziałów: 0.4711543168298382
odchylenie standardowe MSE dla 10 podziałów: 0.06694789041737687

średni RMSE dla 10 podziałów: 0.6844987776738278
odchylenie standardowe RMSE dla 10 podziałów: 0.05114430753147296

średni MAE dla 10 podziałów: 0.6650982630051685
odchylenie standardowe MAE dla 10 podziałów: 0.054806053692890086


# No wrist

In [ ]:
def merge_sets_cartesian_no_wrist(img_matrix, signature_matrix, image_labels,signature_labels):

  # print(img_matrix[0])
  merged_array2=[]
  new_labels2=[]
  for count, mer in enumerate(signature_matrix):
    for count2, img in enumerate(img_matrix):
      if(image_labels[count2] == signature_labels[count]):
        merged_array2.append([img_matrix[count2], signature_matrix[count]])
        new_labels2.append(image_labels[count2])
  # print(np.array(merged_array).shape)
  # print(merged_array[0])
  return merged_array2,new_labels2

In [ ]:
def merge_sets_simple_no_wrist(img_matrix,signature_matrix,image_labels,signature_labels):
  merged_array=[]
  new_labels=[]
  for count ,sig in enumerate(signature_matrix):
    for count2, img in enumerate(img_matrix):
      if(image_labels[count2]==signature_labels[count]):

        merged_array.append([img_matrix[count2], signature_matrix[count]])
        image_labels=np.delete(image_labels, count2, 0)
        img_matrix=np.delete(img_matrix, count2, 0)
        new_labels.append(signature_labels[count])
        break
  return merged_array,new_labels

In [ ]:
def split_sets_back_no_wrist(merged_matrix):
    img_matrix=[]
    sig_matrix=[]
    wrist_matrix=[]
    for  it in merged_matrix:
        img_matrix.append(it[0])
        sig_matrix.append(it[1])
    return img_matrix,sig_matrix

In [ ]:
def feature_extraction2_no_wrist():
  root="/content/drive/MyDrive/BIOMETRIA_SEM3_baza_noc_gier/users"
  global labels
  labels = []
  image_matrix = []
  image_labels = []
  wrist_matrix=[]
  signature_features_matrix=[]
  signature_labels=[]
  wrist_labels=[]
  for user in os.listdir(f'{root}'):
      labels.append(user)
      for filename in os.listdir(f'{root}/{user}/faces'):

          img = cv2.imread(f'{root}/{user}/faces/{filename}')
          # print(img.shape)
          img = cv2.resize(img, (120, 120))
          # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          # img = img.flatten()
          image_matrix.append(np.array(img))
          image_labels.append(labels.index(user))
      for filename in os.listdir(f'{root}/{user}/signatures'):
          signature=read_signature(f'{root}/{user}/signatures/{filename}')
          # print(img.shape)
          stats = get_stats(signature)
          if stats[1]>0 and stats[0]<1:
              signature_features_matrix.append(stats)
              signature_labels.append(labels.index(user))
      # for filename in os.listdir(f'{root}/{user}/wrist_gyroscope'):
      #     # print(f'{root}/{user}/wrist_gyroscope/{filename}')
      #     wrist_matrix.append(read_wrist_csv2(f'{root}/{user}/wrist_gyroscope/{filename}'))
      #     # print(img.shape)
      #     wrist_labels.append(labels.index(user))
         
  IMG_SIZE = (120, 120)
  IMG_SHAPE = IMG_SIZE + (3,)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
  preprocess_input = tf.keras.applications.vgg16.preprocess_input
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  inputs = tf.keras.Input(shape=(120,120, 3))
  x = preprocess_input(inputs)
  x = base_model(x, training=False)
  outputs = global_average_layer(x)
  model = tf.keras.Model(inputs, outputs)

  image_matrix=model.predict(np.array(image_matrix))
  global seed
  train_sig_X, train_sig_Y, test_sig_X, test_sig_Y = split_mono_set(signature_features_matrix, signature_labels,seed)
  train_img_X, train_img_Y, test_img_X, test_img_Y = split_mono_set(image_matrix, image_labels,seed)
  
  return train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, train_img_X, train_img_Y, test_img_X, test_img_Y

In [ ]:
def transformations_concatenation_no_wrist(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                     train_img_X, train_img_Y, test_img_X, test_img_Y):
  pca = PCA(svd_solver="randomized",n_components=comp_img, whiten=True)
  # scaler_img = StandardScaler()
  train_img_X = pca.fit_transform(np.array(train_img_X))
  # train_img_X = scaler_img.fit_transform(np.array(train_img_X))
  # print(train_img_X[0])

  scaler_sig = StandardScaler() #
  train_sig_X=scaler_sig.fit_transform(train_sig_X)
  pca_sig = PCA(svd_solver="randomized", n_components=comp_sig, whiten=True)
  train_sig_X=pca_sig.fit_transform(train_sig_X)
  # train_sig_X=np.fliplr(train_sig_X)



  test_img_X=pca.transform(np.array(test_img_X))
  # test_img_X = scaler_img.transform(np.array(train_img_X))

  test_sig_X = scaler_sig.transform(np.array(test_sig_X))
  test_sig_X = pca_sig.transform(test_sig_X)
  # test_sig_X = np.fliplr(test_sig_X)


  merged_trainX, merged_trainY = merge_sets_cartesian_no_wrist(train_img_X, train_sig_X, train_img_Y,
                                                      train_sig_Y)
  train_img_X, train_sig_X = split_sets_back_no_wrist(merged_trainX)

  merged_testX, merged_testY = merge_sets_simple_no_wrist(test_img_X, test_sig_X, test_img_Y,
                                                      test_sig_Y)
  test_img_X, test_sig_X = split_sets_back_no_wrist(merged_testX)

  trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X)),axis=1)
  trainY=merged_trainY
  testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X)),axis=1)
  testY=merged_testY


    #####
  if mode == 'face':
    trainX=np.array(train_img_X)
    trainY=merged_trainY
    testX=np.array(test_img_X)
    testY=merged_testY
  elif mode == 'sig':
    trainX=np.array(train_sig_X)
    trainY=merged_trainY
    testX=np.array(test_sig_X)
    testY=merged_testY
  elif mode == 'face_sig':
    trainX=np.concatenate((np.array(train_img_X),np.array(train_sig_X)),axis=1)
    trainY=merged_trainY
    testX=np.concatenate((np.array(test_img_X),np.array(test_sig_X)),axis=1)
    testY=merged_testY

    #####
  return trainX, trainY, testX, testY

# No wrist NB tests

In [ ]:
seed=24
decision_threshold=0.6
mode='sig'
# comp_discard=3
comp_img=24
comp_sig=12
comp_wrist=4

In [ ]:
f1_arr = []
MSE_arr = []
RMSE_arr = []
MAE_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y)=feature_extraction2_no_wrist()
  (trainX, trainY, testX, testY)=transformations_concatenation_no_wrist(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y)
  class_model,res=training_testing_report2(trainX, trainY, testX, testY)
  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

In [ ]:
evaluation_results_print()


liczba rekordów w zbiorze treningowym: 2093
liczba rekordów w zbiorze testowym: 44

średni f1-score dla 10 podziałów: 0.7793164200140945
odchylenie standardowe f1-score dla 10 podziałów: 0.0611785651227693

średni MSE dla 10 podziałów: 0.19883401526767924
odchylenie standardowe MSE dla 10 podziałów: 0.042705255691088094

średni RMSE dla 10 podziałów: 0.44304588393554567
odchylenie standardowe RMSE dla 10 podziałów: 0.05044164941246656

średni MAE dla 10 podziałów: 0.23219146157298218
odchylenie standardowe MAE dla 10 podziałów: 0.04143805792529576


# No wrist SVC tests

In [ ]:
seed=24
decision_threshold=0.6
# comp_discard=3
comp_img=24
comp_sig=12
comp_wrist=4

In [ ]:
f1_arr = []
MSE_arr = []
RMSE_arr = []
MAE_arr = []
test_count=10
for i in range(test_count):
  seed+=1
  (train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
  train_img_X, train_img_Y, test_img_X, test_img_Y)=feature_extraction2_no_wrist()
  (trainX, trainY, testX, testY)=transformations_concatenation_no_wrist(train_sig_X, train_sig_Y, test_sig_X, test_sig_Y, 
                                  train_img_X, train_img_Y, test_img_X, test_img_Y)
  class_model,res=training_testing_report1(trainX, trainY, testX, testY)
  f1_arr.append(res)
  MSE_arr.append(MSE(class_model, testX, testY))
  RMSE_arr.append(RMSE(class_model, testX, testY))
  MAE_arr.append(MAE(class_model, testX, testY))

In [ ]:
evaluation_results_print()

liczba rekordów w zbiorze treningowym: 2093
liczba rekordów w zbiorze testowym: 44

średni f1-score dla 10 podziałów: 0.9262634652169535
odchylenie standardowe f1-score dla 10 podziałów: 0.022618279132417436

średni MSE dla 10 podziałów: 0.18275861528460943
odchylenie standardowe MSE dla 10 podziałów: 0.024954596164060077

średni RMSE dla 10 podziałów: 0.4265290259708253
odchylenie standardowe RMSE dla 10 podziałów: 0.02883756732091819

średni MAE dla 10 podziałów: 0.3340543296544802
odchylenie standardowe MAE dla 10 podziałów: 0.02842101789492631
